## Prepare environment

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Load data

In [ ]:
!pip install GitPython

In [ ]:
from langchain.document_loaders import GitLoader # https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/git.html

In [ ]:
loader = GitLoader(repo_path="./data/ios-example-lt64kB/TuningFork", branch="master", file_filter=lambda file_path: file_path.endswith(".swift") or file_path.endswith(".h") or file_path.endswith(".m"))

In [ ]:
data = loader.load()

In [ ]:
len(data)

## Split text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  texts = text_splitter.split_documents(documents)
  return texts

texts = split_docs(data)
print(len(texts))

## Get embeddings

In [ ]:
import os
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
print(embeddings)

## Create index

In [ ]:
import pinecone

In [ ]:
pinecone.init(
    api_key=os.getenv('PINECODE_API_KEY'),
    environment=os.getenv('PINECODE_ENVIRONMENT')
)

In [ ]:
# Check existing indexes
print(pinecone.list_indexes())

In [ ]:
index_name = "ai-repo-reader-1"

In [ ]:
# Clean up previous index before creating new one
pinecone.delete_index(index_name)

In [ ]:
pinecone.create_index(index_name, dimension=1536)

In [ ]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "What does the Tuner class do?"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)

In [ ]:
print(docs[2].page_content)

In [ ]:
query = "How to use the device's microphone to interpret frequency?"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)

In [ ]:
query = "What does the delegate of the Tuner class do?"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)

In [ ]:
print(docs[0].page_content)

In [ ]:
query = "Show me all code extracts where AKFrequencyTracker is used"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)

In [ ]:
print(docs[0].page_content)